***GENERATED CODE FOR breastcancerclassify PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'diagnosis', 'transformation_label': 'String Indexer'}], 'feature': 'diagnosis', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'B', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'diagnosis'}, {'feature_label': 'diagnosis', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('diagnosis')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run breastcancerclassifyHooks.ipynb
try:
	#sourcePreExecutionHook()

	breastcancer = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/breast-cancer.csv', 'filename': 'breast-cancer.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(breastcancer)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run breastcancerclassifyHooks.ipynb
try:
	#transformationPreExecutionHook()

	breastcancerclassifyautofe = TransformationMain.run(breastcancer,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "32630491.12", "stddev": "132693263.17", "min": "8670", "max": "911320502", "missing": "0"}, "updatedLabel": "id"}, {"transformationsData": [{"feature_label": "diagnosis", "transformation_label": "String Indexer"}], "feature": "diagnosis", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "diagnosis"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "radius_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "14.22", "stddev": "3.48", "min": "6.981", "max": "28.11", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "radius_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "texture_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "19.09", "stddev": "4.16", "min": "9.71", "max": "39.28", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "texture_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "perimeter_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "92.61", "stddev": "23.98", "min": "43.79", "max": "188.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "perimeter_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "area_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "662.84", "stddev": "349.36", "min": "143.5", "max": "2501.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "area_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "smoothness_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.1", "stddev": "0.01", "min": "0.06251", "max": "0.1447", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "smoothness_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "compactness_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.1", "stddev": "0.05", "min": "0.01938", "max": "0.3454", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "compactness_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "concavity_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.09", "stddev": "0.08", "min": "0.0", "max": "0.4268", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "concavity_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "concave points_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.05", "stddev": "0.04", "min": "0.0", "max": "0.2012", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "concave points_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "symmetry_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.18", "stddev": "0.03", "min": "0.1167", "max": "0.304", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "symmetry_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fractal_dimension_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.06", "stddev": "0.01", "min": "0.04996", "max": "0.09744", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fractal_dimension_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "radius_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.41", "stddev": "0.28", "min": "0.1115", "max": "2.873", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "radius_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "texture_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.2", "stddev": "0.54", "min": "0.3602", "max": "4.885", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "texture_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "perimeter_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.9", "stddev": "2.05", "min": "0.757", "max": "21.98", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "perimeter_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "area_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "41.13", "stddev": "46.69", "min": "6.802", "max": "542.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "area_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "smoothness_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.0", "min": "0.001713", "max": "0.03113", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "smoothness_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "compactness_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.03", "stddev": "0.02", "min": "0.002252", "max": "0.1354", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "compactness_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "concavity_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.03", "stddev": "0.03", "min": "0.0", "max": "0.396", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "concavity_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "concave points_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.01", "min": "0.0", "max": "0.05279", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "concave points_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "symmetry_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.02", "stddev": "0.01", "min": "0.007882", "max": "0.07895", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "symmetry_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fractal_dimension_se", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "8.948E-4", "max": "0.02984", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fractal_dimension_se"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "radius_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "16.42", "stddev": "4.81", "min": "7.93", "max": "36.04", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "radius_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "texture_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "25.51", "stddev": "6.06", "min": "12.02", "max": "49.54", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "texture_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "perimeter_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "108.26", "stddev": "33.31", "min": "50.41", "max": "251.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "perimeter_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "area_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "896.0", "stddev": "571.07", "min": "185.2", "max": "4254.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "area_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "smoothness_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.13", "stddev": "0.02", "min": "0.07117", "max": "0.2226", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "smoothness_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "compactness_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.26", "stddev": "0.16", "min": "0.02729", "max": "1.058", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "compactness_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "concavity_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.28", "stddev": "0.21", "min": "0.0", "max": "1.252", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "concavity_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "concave points_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.12", "stddev": "0.07", "min": "0.0", "max": "0.291", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "concave points_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "symmetry_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.29", "stddev": "0.06", "min": "0.1565", "max": "0.6638", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "symmetry_worst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fractal_dimension_worst", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.08", "stddev": "0.02", "min": "0.05504", "max": "0.2075", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fractal_dimension_worst"}]}))

	#transformationPostExecutionHook(breastcancerclassifyautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run breastcancerclassifyHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(breastcancerclassifyautofe, ["id", "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean", "fractal_dimension_mean", "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se", "compactness_se", "concavity_se", "concave points_se", "symmetry_se", "fractal_dimension_se", "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst", "compactness_worst", "concavity_worst", "concave points_worst", "symmetry_worst", "fractal_dimension_worst"], "diagnosis_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

